## Create graphs of population development from 1997 to 2019 in 500 meters range from stations

Reads stations_population, railways and stations data and creates map figure for every year. Result is saved in folder pics.

#### Import libraries

In [11]:
from pyproj import CRS
import geopandas as gpd
from functools import reduce
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import contextily as ctx

#### Read population, rails, subway stations and rail stations data

In [12]:
# Read rails data
fp = "railway_data/railways.shp"
rails = gpd.read_file(fp)

# Read stations population data
fp = "population_data/stations_population.shp"
stations_population = gpd.read_file(fp)

# Read stations data
fp = "stations_data/stations.shp"
stations = gpd.read_file(fp)

#### Set coordinate reference system

In [13]:
# Rails
rails = rails.to_crs(epsg=3857)

# Population all years
stations_population = stations_population.to_crs(epsg=3857)

# Stations
stations = stations.to_crs(epsg=3857)

# Check that they all have same CRS
print("All have same CRS: ", rails.crs == stations_population.crs == stations.crs)

All have same CRS:  True


#### Create years_list for the loop

In [14]:
# First get columns names
columns = stations_population.columns.tolist()

# Then filter columns list and leave only population columns (they have year eg. pop1997)
pop_years = list(filter(lambda x: 'pop' in x, columns))

# Remove pop and leave only years. Then years_list is ready for loop.
years_list=[]
for name in pop_years:
    year = name[3:7]
    year = int(year)
    years_list.append(year)
    
# Print years_list
print("Years: \n", years_list)

Years: 
 [1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]


#### Create and save annual images

In [15]:
# Loop through years and create and save figures
for year in years_list:
    
    # Select colour map
    cmap = matplotlib.cm.get_cmap('autumn_r')
    
    # Set column name which to visualize
    column_name = "pop"+str(year)
    
    # Define variable total population and change type to integer
    total_population = int(round(stations_population[column_name].sum()))
    
    # set the range for the choropleth
    vmin, vmax = 0, 15000

    # create figure and axes for Matplotlib
    fig, ax = plt.subplots(1, figsize=(15, 10))

    # Plot the population data
    stations_population.plot(ax=ax, column=column_name, cmap=cmap, linewidth=0.5, alpha=0.8, norm=plt.Normalize(vmin=vmin, vmax=vmax))
    
    # Plot the rails
    rails.plot(ax=ax, color="gray", linewidth=0.5)
    
    # Plot stations
    stations.plot(ax=ax, color="black", markersize=10)

    # remove the axis
    ax.axis("off")

    # add a title
    ax.set_title("Year " +str(year) + " - Population living in 500m range from station " + str(total_population), 
                 fontdict={"fontsize": "20", "fontweight" : "3"})

    # Add source to the bottom left corner
    ax.annotate("Source: HSY.fi, Väestöruudukko, 2020",xy=(0.1, .08), xycoords="figure fraction", 
                horizontalalignment="left", verticalalignment="top", fontsize=12, color="#555555")
    
    # Create colorbar as a legend
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=vmin, vmax=vmax))

    # empty array for the data range
    sm._A = []

    # Add the colorbar to the figure
    cbar = fig.colorbar(sm)

    # Add basemap with `OSM_A` style
    ctx.add_basemap(ax, url=ctx.tile_providers.OSM_A)

    # Crop the figure
    ax.set_xlim(2725000, 2816000)
    ax.set_ylim(8420000, 8490000)

    #saving our map as .png file.
    fig.savefig("pics/stations_"+column_name+".png", dpi=100)
    
    # Close figure. Remove if you want to see figures on notepad.
    plt.close(fig)